<a href="https://colab.research.google.com/github/yagneshs2004/deeplearning/blob/main/brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import random
import os
import shutil

In [2]:
source_path = '/content/drive/MyDrive/brain_tumor_dataset2'
tumor_path = '/content/drive/MyDrive/brain_tumor_dataset2/yes'
no_tumor_path = '/content/drive/MyDrive/brain_tumor_dataset2/no'

In [3]:
print(os.listdir(source_path))
print(os.listdir(tumor_path))
print(os.listdir(no_tumor_path))

['yes', 'no', '.ipynb_checkpoints']
['Y157.JPG', 'Y111.JPG', 'Y104.jpg', 'Y156.JPG', 'Y109.JPG', 'Y106.jpg', 'Y102.jpg', 'Y147.JPG', 'Y13.jpg', 'Y108.jpg', 'Y112.JPG', 'Y116.JPG', 'Y146.JPG', 'Y10.jpg', 'Y115.JPG', 'Y155.JPG', 'Y153.jpg', 'Y114.JPG', 'Y107.jpg', 'Y113.JPG', 'Y12.jpg', 'Y15.jpg', 'Y1.jpg', 'Y117.JPG', 'Y148.JPG', 'Y105.jpg', 'Y11.jpg', 'Y103.jpg', 'Y100.JPG', 'Y101.jpg', 'Y14.jpg', 'Y154.jpg', 'Y120.JPG', 'Y165.JPG', 'Y251.JPG', 'Y242.JPG', 'Y19.JPG', 'Y246.JPG', 'Y26.jpg', 'Y159.JPG', 'Y184.JPG', 'Y3.jpg', 'Y17.jpg', 'Y192.JPG', 'Y160.JPG', 'Y253.JPG', 'Y18.JPG', 'Y25.jpg', 'Y162.jpg', 'Y259.JPG', 'Y195.JPG', 'Y22.jpg', 'Y254.jpg', 'Y29.jpg', 'Y28.jpg', 'Y187.jpg', 'Y193.JPG', 'Y182.JPG', 'Y250.jpg', 'Y247.JPG', 'Y243.JPG', 'Y24.jpg', 'Y256.JPG', 'Y258.JPG', 'Y249.JPG', 'Y252.jpg', 'Y161.JPG', 'Y257.jpg', 'Y20.jpg', 'Y244.JPG', 'Y16.JPG', 'Y194.jpg', 'Y2.jpg', 'Y167.JPG', 'Y168.jpg', 'Y255.JPG', 'Y170.JPG', 'Y186.jpg', 'Y248.JPG', 'Y185.jpg', 'Y180.jpg', 'Y188.jpg', 'Y

In [4]:
root_path = 'content/dataset/'

In [5]:
training = os.path.join(root_path,'training')
testing = os.path.join(root_path,'testing')
train_tumor = os.path.join(training, 'tumor')
test_tumor = os.path.join(testing, 'tumor')
train_no_tumor = os.path.join(training, 'no_tumor')
test_no_tumor = os.path.join(testing,'no_tumor')

In [6]:
os.makedirs(train_tumor)
os.makedirs(test_tumor)
os.makedirs(train_no_tumor)
os.makedirs(test_no_tumor)

In [7]:
split_size = 0.9

In [8]:
def data_copy(source, train_dest, test_dest, size):
  files = os.listdir(source)
  files = random.sample(files,len(files))
  for i, n in enumerate(files):
    if os.path.getsize(os.path.join(source, n)) <= 0:
      print(n,"is zero length, so ignoring.")
    elif i<size*len(files):
      shutil.copyfile(os.path.join(source,n),os.path.join(train_dest,n))
    else:
      shutil.copyfile(os.path.join(source,n),os.path.join(test_dest,n))

In [9]:
data_copy(tumor_path,train_tumor,test_tumor,split_size)
data_copy(no_tumor_path,train_no_tumor,test_no_tumor,split_size)

In [10]:
from PIL import Image

In [13]:
target_size = (150, 150)  # Example target size

# Function to resize images in a directory
def resize_images(directory, target_size):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.JPG') or file.endswith('.JPEG') or file.endswith('.jpeg'):  # You can specify other extensions if needed
                filepath = os.path.join(root, file)
                try:
                    image = Image.open(filepath)
                    resized_image = image.resize(target_size, Image.ANTIALIAS)
                    resized_image.save(filepath)
                except Exception as e:
                    image = image.convert('RGB')
                    image.save(filepath)
                    resized_image = image.resize(target_size, Image.ANTIALIAS)
                    resized_image.save(filepath)

# Resize images in the dataset directory


In [14]:
resize_images(train_tumor, target_size)
resize_images(train_no_tumor, target_size)
resize_images(test_tumor, target_size)
resize_images(test_no_tumor, target_size)

<ipython-input-13-49990f4d982a>:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize(target_size, Image.ANTIALIAS)
<ipython-input-13-49990f4d982a>:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize(target_size, Image.ANTIALIAS)


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
datagen_train = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
datagen_test = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

In [17]:
train_generator = datagen_train.flow_from_directory(directory=training,
                                                                batch_size=20,
                                                                class_mode='binary',
                                                                target_size=(150, 150))

Found 229 images belonging to 2 classes.


In [18]:
test_generator = datagen_test.flow_from_directory(directory=testing,
                                                                batch_size=20,
                                                                class_mode='binary',
                                                                target_size=(150, 150))

Found 24 images belonging to 2 classes.


In [21]:
import cv2

In [25]:
import matplotlib.pyplot as plt

In [54]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(128,(3,3),input_shape=(150,150,3),activation='relu'),
      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation="relu"),
      tf.keras.layers.Dense(1, activation="sigmoid")
  ])


In [55]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=test_generator)

Epoch 1/15
12/12 [==============================] - 75s 6s/step - loss: 2.1521 - accuracy: 0.5459 - val_loss: 0.6924 - val_accuracy: 0.6250
Epoch 2/15
12/12 [==============================] - 73s 6s/step - loss: 0.6922 - accuracy: 0.6114 - val_loss: 0.6915 - val_accuracy: 0.6250
Epoch 3/15
12/12 [==============================] - 75s 6s/step - loss: 0.6914 - accuracy: 0.6114 - val_loss: 0.6907 - val_accuracy: 0.6250
Epoch 4/15
 9/12 [=====================>........] - ETA: 17s - loss: 0.6907 - accuracy: 0.6095